In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Téléchargement de la base de données

In [2]:
# Pour récupérer le nombre de classes du training dataset
from tensorflow import keras
from tensorflow.keras import layers

In [3]:
!git clone https://github.com/fabiopereira59/abeilles-cap500

Cloning into 'abeilles-cap500'...
remote: Enumerating objects: 24878, done.
remote: Counting objects: 100% (6126/6126), done.
remote: Compressing objects: 100% (6121/6121), done.
remote: Total 24878 (delta 2), reused 6126 (delta 2), pack-reused 18752
Receiving objects: 100% (24878/24878), 242.93 MiB | 16.59 MiB/s, done.
Resolving deltas: 100% (5/5), done.
Checking out files: 100% (24081/24081), done.


In [4]:
IMG_SIZE = 224
train_ds = keras.utils.image_dataset_from_directory(
    directory='abeilles-cap500/train/',
    labels='inferred',
    label_mode='categorical',
    shuffle = False,
    batch_size=16,
    image_size=(IMG_SIZE, IMG_SIZE))

Found 14917 files belonging to 71 classes.


In [5]:
CLASS_NAMES = train_ds.class_names
print(CLASS_NAMES)
NB_CLASSES = len(CLASS_NAMES)
print(NB_CLASSES)

['Amegilla quadrifasciata', 'Andrena agilissima', 'Andrena bicolor', 'Andrena cineraria', 'Andrena clarkella', 'Andrena denticulata', 'Andrena flavipes', 'Andrena florea', 'Andrena fulva', 'Andrena gravida', 'Andrena haemorrhoa', 'Andrena hattorfiana', 'Andrena nigroaenea', 'Andrena nitida', 'Andrena nycthemera', 'Andrena thoracica', 'Andrena vaga', 'Andrena ventralis', 'Anthidiellum strigatum', 'Anthidium florentinum', 'Anthidium manicatum', 'Anthidium oblongatum', 'Anthidium septemspinosum', 'Anthophora bimaculata', 'Anthophora furcata', 'Anthophora plumipes', 'Apis mellifera', 'Bombus argillaceus', 'Bombus bohemicus', 'Bombus campestris', 'Bombus hortorum', 'Bombus humilis', 'Bombus hypnorum', 'Bombus lapidarius', 'Bombus lucorum', 'Bombus muscorum', 'Bombus pascuorum', 'Bombus pratorum', 'Bombus ruderatus', 'Bombus rupestris', 'Bombus sylvarum', 'Bombus terrestris lusitanicus', 'Bombus vestalis', 'Ceratina cucurbitina', 'Chelostoma florisomne', 'Colletes cunicularius', 'Colletes he

## Chargement des données

In [6]:
from tensorflow import keras
from tensorflow.keras import layers

In [7]:
# Paramètres
IMG_SIZE = 224 # pour utiliser ResNet

In [8]:
# Récupération des dataset pour l'entraînement (train, val)
train_ds = keras.utils.image_dataset_from_directory(
    directory='abeilles-cap500/train/',
    labels='inferred',
    label_mode='categorical',
    shuffle = False,
    batch_size=16,
    image_size=(IMG_SIZE, IMG_SIZE))

validation_ds = keras.utils.image_dataset_from_directory(
    directory='abeilles-cap500/val/',
    labels='inferred',
    label_mode='categorical',
    batch_size=16,
    image_size=(IMG_SIZE, IMG_SIZE))

Found 14917 files belonging to 71 classes.
Found 1832 files belonging to 71 classes.


In [9]:
len(train_ds.file_paths)

14917

## Création du modèle

In [10]:
!pip uninstall opencv-python-headless==4.5.5.62
!pip install opencv-python-headless==4.1.2.30
!pip install -q -U albumentations
!echo "$(pip freeze | grep albumentations) is successfully installed"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 21.8 MB 1.3 MB/s 
     |████████████████████████████████| 116 kB 34.9 MB/s 
albumentations==1.2.1 is successfully installed


In [11]:
from albumentations import (Compose, Rotate, HorizontalFlip, VerticalFlip, Affine, RandomBrightnessContrast, ChannelShuffle)
import albumentations as A

AUGMENTATIONS_TRAIN = Compose([
    Rotate(limit=[0,100], p=0.5),
    HorizontalFlip(p=0.5),
    VerticalFlip(p=0.5),
    Affine(shear=[-45, 45], p=0.5),
    RandomBrightnessContrast(p=0.5)
])

In [12]:
import numpy as np

x_train = np.array(train_ds.file_paths)
y_train = np.zeros((14917, NB_CLASSES))

ind_data = 0
for bx, by in train_ds.as_numpy_iterator():
  y_train[ind_data:ind_data+bx.shape[0]] = by
  ind_data += bx.shape[0]

In [13]:
def color_preprocessing(x):
    x = x.astype('float32')
    mean = [125.3, 123.0, 113.9]
    std  = [63.0,  62.1,  66.7]
    for i in range(3):
        x[:,:,:,i] = (x[:,:,:,i] - mean[i]) / std[i]
    return x

In [14]:
from tensorflow.keras.utils import Sequence
import numpy as np
import cv2 as cv

class AbeillesSequence(Sequence):
    # Initialisation de la séquence avec différents paramètres
    def __init__(self, x_train, y_train, batch_size, augmentations):
        self.x_train = x_train
        self.y_train = y_train
        self.classes = CLASS_NAMES
        self.batch_size = batch_size
        self.augment = augmentations
        self.indices1 = np.arange(len(x_train))
        np.random.shuffle(self.indices1) # Les indices permettent d'accéder
        # aux données et sont randomisés à chaque epoch pour varier la composition
        # des batches au cours de l'entraînement

    # Fonction calculant le nombre de pas de descente du gradient par epoch
    def __len__(self):
        return int(np.ceil(x_train.shape[0] / float(self.batch_size)))
    
    # Application de l'augmentation de données à chaque image du batch
    def apply_augmentation(self, bx, by):

        batch_x = np.zeros((bx.shape[0], IMG_SIZE, IMG_SIZE, 3))
        batch_y = by
        
        # Pour chaque image du batch
        for i in range(len(bx)):
            class_labels = []
            class_id = np.argmax(by[i])
            class_labels.append(self.classes[class_id])

            # Application de l'augmentation à l'image et aux masques
            img = cv.imread(bx[i])
            img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
            transformed = self.augment(image=img)
            batch_x[i] = transformed['image']
            #print(batch_x[i])
      
        return batch_x, batch_y

    # Fonction appelée à chaque nouveau batch : sélection et augmentation des données
    # idx = position du batch (idx = 5 => on prend le 5ème batch)
    def __getitem__(self, idx):
        batch_x = self.x_train[self.indices1[idx * self.batch_size:(idx + 1) * self.batch_size]]
        batch_y = self.y_train[self.indices1[idx * self.batch_size:(idx + 1) * self.batch_size]]
           
        batch_x, batch_y = self.apply_augmentation(batch_x, batch_y)

        # Normalisation des données
        batch_x = color_preprocessing(batch_x)
        
        return batch_x, batch_y

    # Fonction appelée à la fin d'un epoch ; on randomise les indices d'accès aux données
    def on_epoch_end(self):
        np.random.shuffle(self.indices1)

In [15]:
from tensorflow.keras import regularizers
from tensorflow.keras import optimizers
import tensorflow as tf

In [16]:
from tensorflow import keras 
import numpy as np
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import BatchNormalization
from keras.layers import Conv2D, Dense, Input, add, Activation, Flatten, AveragePooling2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.callbacks import LearningRateScheduler, TensorBoard
from keras.regularizers import l2
from keras import optimizers
from keras.models import Model

WEIGHT_DECAY = 0.0005

def create_WideResNet50(width=1, img_rows=224, img_cols=224, img_channels=3, nb_classes=1000):

  # Input
  img_input = Input(shape=(img_rows,img_cols,img_channels))

  # first_filter : nb de filtres de la première conv
  # reduction : True si on doit diviser la dimension de l'entrée
  def residual_layer(input, first_filter, reduction=False, first_layer=False):

    conv = Conv2D(filters=width*first_filter, kernel_size=(1,1), strides=(1,1),
        kernel_initializer='he_normal',
        kernel_regularizer=l2(WEIGHT_DECAY))(input)
    conv = BatchNormalization(momentum=0.9, epsilon=1e-5)(conv)
    conv = Activation('relu')(conv)

    if (reduction):
      conv = Conv2D(filters=width*first_filter, kernel_size=(3,3), strides=(2,2), padding='same',
              kernel_initializer='he_normal',
              kernel_regularizer=l2(WEIGHT_DECAY))(conv)
    else:
      conv = Conv2D(filters=width*first_filter, kernel_size=(3,3), strides=(1,1), padding='same',
              kernel_initializer='he_normal',
              kernel_regularizer=l2(WEIGHT_DECAY))(conv)
    conv = BatchNormalization(momentum=0.9, epsilon=1e-5)(conv)
    conv = Activation('relu')(conv)

    conv = Conv2D(filters=4*first_filter, kernel_size=(1,1), strides=(1,1),
            kernel_initializer='he_normal',
            kernel_regularizer=l2(WEIGHT_DECAY))(conv)
    conv = BatchNormalization(momentum=0.9, epsilon=1e-5)(conv)

    if (first_layer):
      conv2 = Conv2D(filters = 256, kernel_size=(1,1), strides=(1,1),
          kernel_initializer='he_normal',
          kernel_regularizer=l2(WEIGHT_DECAY))(x)
      conv2 = BatchNormalization(momentum=0.9, epsilon=1e-5)(conv2)
    else:
      if (reduction):
        conv2 = Conv2D(filters = 4*first_filter, kernel_size=(1,1), strides=(2,2),
                kernel_initializer='he_normal',
                kernel_regularizer=l2(WEIGHT_DECAY))(x)
        conv2 = BatchNormalization(momentum=0.9, epsilon=1e-5)(conv2)
      else:
        conv2 = input
    
    return add([conv, conv2])
  
  x = ZeroPadding2D(padding=(3,3))(img_input)
  x = Conv2D(filters=64, kernel_size=(7,7), strides=(2,2),
        kernel_initializer='he_normal',
        kernel_regularizer=l2(WEIGHT_DECAY))(x)
  x = BatchNormalization(momentum=0.9, epsilon=1e-5)(x)
  x = Activation('relu')(x)
  x = ZeroPadding2D(padding=(1,1))(x)
  x = MaxPooling2D(pool_size=(3, 3), strides=(2,2))(x)

  # Premier bloc résiduel
  x = residual_layer(x, 64, reduction=False, first_layer=True)
  for i in range(2): 
    x = Activation('relu')(x)
    x = residual_layer(x, 64, reduction=False)

  # Second bloc
  x = Activation('relu')(x)
  x = residual_layer(x, 128, reduction=True)
  for i in range(3): 
    x = Activation('relu')(x)
    x = residual_layer(x, 128, reduction=False)
  
  # Troisième bloc
  x = Activation('relu')(x)
  x = residual_layer(x, 256, reduction=True)
  for i in range(5): 
    x = Activation('relu')(x)
    x = residual_layer(x, 256, reduction=False)

  # Quatrième bloc
  x = Activation('relu')(x)
  x = residual_layer(x, 512, reduction=True)
  for i in range(2): 
    x = Activation('relu')(x)
    x = residual_layer(x, 512, reduction=False)

  x = GlobalAveragePooling2D()(x)
  x = Flatten()(x)
  x = Dense(nb_classes,
      activation='softmax',
      kernel_initializer='he_normal',
      kernel_regularizer=l2(WEIGHT_DECAY))(x)
  
  return Model(img_input, x)

In [20]:
model = create_WideResNet50(width=2, img_rows=224, img_cols=224, img_channels=3, nb_classes=NB_CLASSES)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d_2 (ZeroPadding2  (None, 230, 230, 3)  0          ['input_2[0][0]']                
 D)                                                                                               
                                                                                                  
 conv2d_53 (Conv2D)             (None, 112, 112, 64  9472        ['zero_padding2d_2[0][0]']       
                                )                                                           

In [21]:
# Ajout de l'optimiseur, de la fonction coût et des métriques
model.compile(tf.keras.optimizers.SGD(learning_rate=1e-3, momentum=0.9), loss='categorical_crossentropy', metrics=['categorical_accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

In [ ]:
def weight_class_i(nb_elts, nb_class, nb_img):
  return (nb_img/(nb_class*nb_elts))

In [ ]:
# Détermination des poids à attribuer à chaque classe
import pathlib

nb_files = 0
class_weight = {}
for i in range(nb_classes):
  for path in pathlib.Path("abeilles-cap500/train/" + class_names[i]).iterdir():
    if path.is_file():
        nb_files += 1
  class_weight[i] = weight_class_i(nb_files, nb_classes, len(train_ds.file_paths))
  nb_files = 0
print(class_weight)

{0: 1.265654165959613, 1: 1.8758802816901408, 2: 2.414926339647078, 3: 0.5252464788732394, 4: 1.0452666246233622, 5: 2.918035993740219, 6: 1.029895056614195, 7: 1.2215034392400916, 8: 0.5252464788732394, 9: 2.7285531370038414, 10: 0.5252464788732394, 11: 0.6089814247805675, 12: 1.3822275759822091, 13: 1.235874067937034, 14: 2.334428794992175, 15: 3.622389509470617, 16: 0.5252464788732394, 17: 1.9820621844273185, 18: 1.8758802816901408, 19: 0.8471717401181281, 20: 0.6956907004943569, 21: 1.8269442743417024, 22: 1.842970101309612, 23: 2.0397921509640367, 24: 2.659475842396149, 25: 0.5602629107981221, 26: 0.5252464788732394, 27: 1.4006572769953052, 28: 1.654319618498392, 29: 1.207463169823539, 30: 0.7928248737709275, 31: 1.2733247972684592, 32: 0.5252464788732394, 33: 0.5252464788732394, 34: 0.7781429316640585, 35: 2.7285531370038414, 36: 0.5252464788732394, 37: 0.5252464788732394, 38: 2.8013145539906104, 39: 0.7295089984350548, 40: 0.5968709987195903, 41: 1.4489558037882466, 42: 0.981769

In [25]:
# Les callbacks
model_checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath='./drive/MyDrive/Stage2A/cap500/WideResNet50-2/PoidsWR50-2/best_model_wideresnet50-2',
    save_weights_only=True,
    monitor='val_categorical_accuracy',
    mode='max',
    save_best_only=True,
    verbose=1)

early_stopping_cb = tf.keras.callbacks.EarlyStopping(
    monitor="val_categorical_accuracy",
    min_delta=0.01,
    patience=10,
    verbose=1,
    mode="auto")

reduce_lr_cb = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_categorical_accuracy', factor=0.1,
                              patience=5, min_lr=0.00001, verbose=1)

In [26]:
train_ds_aug = AbeillesSequence(x_train, y_train, batch_size=16, augmentations=AUGMENTATIONS_TRAIN)

In [27]:
import numpy as np
import tensorflow as tf
# Normalisation des données de validation
x_val = np.zeros((1832, IMG_SIZE, IMG_SIZE, 3))
y_val = np.zeros((1832, NB_CLASSES))

ind_data = 0
for bx, by in validation_ds.as_numpy_iterator():
  x_val[ind_data:ind_data+bx.shape[0]] = bx
  y_val[ind_data:ind_data+bx.shape[0]] = by
  ind_data += bx.shape[0]

x_val = color_preprocessing(x_val)

In [ ]:
# Sans pondération des classes
history = model.fit(train_ds_aug, epochs=150, validation_data = (x_val, y_val), callbacks=[model_checkpoint_cb, early_stopping_cb, reduce_lr_cb])
# Avec pondération des classes
#model.fit(train_ds_aug, epochs=150, validation_data = (x_val, y_val), callbacks=[model_checkpoint_cb, early_stopping_cb, reduce_lr_cb], class_weight=class_weight)

Epoch 1/150
933/933 [==============================] - ETA: 0s - loss: 38.2502 - categorical_accuracy: 0.0461 - precision_1: 0.1111 - recall_1: 0.0013
Epoch 1: val_categorical_accuracy improved from -inf to 0.06932, saving model to ./drive/MyDrive/Stage2A/cap500/WideResNet50-2/PoidsWR50-2/best_model_wideresnet50-2
933/933 [==============================] - 394s 399ms/step - loss: 38.2502 - categorical_accuracy: 0.0461 - precision_1: 0.1111 - recall_1: 0.0013 - val_loss: 37.9996 - val_categorical_accuracy: 0.0693 - val_precision_1: 0.1089 - val_recall_1: 0.0060 - lr: 0.0010
Epoch 2/150
933/933 [==============================] - ETA: 0s - loss: 37.4635 - categorical_accuracy: 0.0659 - precision_1: 0.1609 - recall_1: 0.0025
Epoch 2: val_categorical_accuracy improved from 0.06932 to 0.07424, saving model to ./drive/MyDrive/Stage2A/cap500/WideResNet50-2/PoidsWR50-2/best_model_wideresnet50-2
933/933 [==============================] - 376s 403ms/step - loss: 37.4635 - categorical_accuracy: 0.

In [ ]:
import matplotlib.pyplot as plt
def plot_training_analysis(history, metric='loss'):    

  loss = history.history[metric]
  val_loss = history.history['val_' + metric]

  epochs = range(len(loss))

  plt.plot(epochs, loss, 'b', linestyle="--",label='Training ' + metric)
  plt.plot(epochs, val_loss, 'g', label='Validation ' + metric)
  plt.title('Training and validation ' + metric)
  plt.legend()

  plt.show()

In [ ]:
plot_training_analysis(history)

NameError: ignored